**1_1_numpy_tensorflow_pytorch.ipynb**




In [ ]:
# comment를 직접 달 수 있다. 
# Pytorch 설치 전 : 런타임 메뉴 > 런타임 유형 변경 > GPU 지정
# Pytorch 설치 (매 세션마다 설치 필요함.)
# 설치하기 : !pip3 install torch 라고 한다.
!pip3 install torch
!pip3 install torchvision


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**패키지 비교**
- Numpy vs Tensorflow vs PyTorch
- output = x * y + z 


In [ ]:
import numpy as np
from datetime import datetime

start = datetime.now()

np.random.seed(0)  # seed(0) 함수는 random 값을 같은 값으로 고정시킴

N, D = 3, 4

x = np.random.randn(N, D) # 3행 4 열 배열에 0 ~ 1 사이의 실수형 랜덤값을 체움
y = np.random.randn(N, D)
z = np.random.randn(N, D)

a = x * y
b = a + z
c = np.sum(b)

grad_c = 1.0
grad_b = grad_c * np.ones((N, D))
grad_a = grad_b.copy()
grad_z = grad_b.copy()
grad_y = grad_a * y 
grad_x = grad_a * x

print(grad_x)
print(grad_y)
print(grad_z)

print("연산 처리 시간 : ", datetime.now() - start)

[[ 1.76405235  0.40015721  0.97873798  2.2408932 ]
 [ 1.86755799 -0.97727788  0.95008842 -0.15135721]
 [-0.10321885  0.4105985   0.14404357  1.45427351]]
[[ 0.76103773  0.12167502  0.44386323  0.33367433]
 [ 1.49407907 -0.20515826  0.3130677  -0.85409574]
 [-2.55298982  0.6536186   0.8644362  -0.74216502]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
연산 처리 시간 :  0:00:00.001987


Tensorflow 는 graph 로 연산을 나타내는 시스템이며, 연산하려면 Session 상에서 실행되어야 한다. 
Session은 graph(연산을 구성하는 노드) 작업을 cpu 나 gpu 같은 device에 배정하고 실행을 위한 메소드들을 제공한다.

In [ ]:
import tensorflow as tf
import numpy as np
from datetime import datetime

# tf.placeholder() 함수 AttributeError 발생시 해결 방법
# tensorflow 2.0 에서는 사용 못 함
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# ===============================

start = datetime.now()

# 실행 동작을 미리 정의해 놓음
with tf.device('/gpu:0'): # 실행시 gpu 0 사용을 지정함.
  x = tf.placeholder(tf.float32) # 나중에 값을 넣을 공간을 만듦
  y = tf.placeholder(tf.float32) 
  z = tf.placeholder(tf.float32)

  a = x * y # x[0][0] 값 * y[0][0], x[0][1] 값 * y[0][1],,..등 이어간다. 
  b = a * z 
  c = tf.reduce_sum(b) # 배열의 모든 값들의 합계를 구함

grad_x, grad_y, grad_z = tf.gradients(c, [x, y, z])

# 준비된 공간의 값을 채우는 설정 
with tf.Session() as sess:
  values = {
      x: np.random.randn(N, D), 
      y: np.random.randn(N, D), 
      z: np.random.randn(N, D)
  }

  out = sess.run([c, grad_x, grad_y, grad_z], feed_dict = values)
  c_val, grad_x_val, grad_y_val, grad_z_val = out

print(grad_x_val)
print(grad_y_val)
print(grad_z_val)

print("연산 처리 시간 : ", datetime.now() - start)


[[ 1.0852824   0.07649145  0.7281453  -0.66790307]
 [-0.09063018  0.47435552  0.04594262  0.1982244 ]
 [-0.06035097  0.01571202 -0.46247828 -0.04678741]]
[[-0.82732177 -0.43231946  0.31495336  0.52185994]
 [-0.18604071  0.57053614  2.781559    0.9027842 ]
 [ 0.4624066  -0.02275594 -0.91340125  0.10028297]]
[[-1.9855634  -0.2557946   0.3468382  -0.1169617 ]
 [ 0.53560627  1.6765189   0.0480865   0.8355793 ]
 [-0.03390065 -0.13250516  0.79467577 -0.28202778]]
연산 처리 시간 :  0:00:02.905485


In [ ]:
import torch
from torch.autograd import Variable
from datetime import datetime

start = datetime.now()

N, D = 3, 4

# 자동 미분 계산 함수 : autograd.Variable
x = Variable(torch.randn(N, D).cuda(), requires_grad=True)
y = Variable(torch.randn(N, D).cuda(), requires_grad=True)
z = Variable(torch.randn(N, D).cuda(), requires_grad=True)

a = x * y
b = a * z
c = torch.sum(b)

# gradient(경사도) 자동 계산 수행
# 기우기(gradient)기ㅏ 1.0이라고 가정하고 최종 값이 c 에서 
# backward를 통해서 역전파를 해 준다. 
c.backward(gradient=torch.tensor(1, dtype=torch.float))

print(x.grad)
print(y.grad)
print(z.grad)

print("연산 처리 시간 : ", datetime.now() - start)



tensor([[ 1.1904, -0.0545, -1.0410, -0.4978],
        [ 1.2901, -0.0502,  0.4260, -0.3523],
        [-0.0348, -0.4158, -0.5879,  0.3134]], device='cuda:0')
tensor([[ 0.1396, -0.0830, -0.7127, -0.5635],
        [-2.4382, -0.1508,  0.4417,  0.4095],
        [-0.3443, -0.0527, -0.4310, -0.2029]], device='cuda:0')
tensor([[ 0.2634,  0.0053,  1.1129,  0.8758],
        [-1.5962,  1.2140,  1.6388, -0.3028],
        [ 0.7414,  0.0376,  5.0417, -0.2378]], device='cuda:0')
연산 처리 시간 :  0:00:00.105026
